In [1]:
import pandas as pd
import numpy as np
import cvxpy as cp
import mosek
import pickle
from pandas.tseries.offsets import BMonthEnd
from time import time

In [2]:
n = 2000

# Get the excess returns data
idx = pd.read_csv('data/TOP{}_universe.csv'.format(n), parse_dates=[0])
idx = idx[idx['date'] >= '1970-01-01'].reset_index(drop = True)

returndata = pd.read_csv('data/crspmsf.csv', index_col = 0)[['date', 'permno', 'ret']]
returndata['date'] = pd.to_datetime( returndata['date'] )
returndata['permno'] = returndata['permno'].astype(int)
returndata = returndata[returndata['date'] >= '1970-01-01'].reset_index(drop = True)

betaData = pd.read_csv('data/return_pf_excess_ret_beta.csv', index_col = 0)[['date','permno' ,'beta_30']]
betaData['date'] = pd.to_datetime(betaData['date'],format='%Y%m') + BMonthEnd()
betaData = betaData[betaData['date'] >= '1970-01-01'].reset_index(drop = True)

with open('data/exalpha_beta_30.pkl', 'rb') as f:
    data = pickle.load(f)

print(idx.head())
print(data.head())
print(returndata.head())

universe = pd.read_csv(f'data/TOP2000_universe.csv',parse_dates=[0],index_col=0)
ff_loadings = pd.read_pickle('data/ffloadings_halflife36_TOP2000.pkl')
ff_cov_df = pd.read_pickle(f'data/ff_cov_halflife{36}.pkl')
spec_est = pd.read_pickle(f'data/spec_est_halflife{36}_TOP{2000}.pkl')

def risk_cov(date):
    ff_cols = ['Mkt-RF','HML','SMB']
    betas = ff_loadings.loc[date,(universe.loc[date],slice(None))].unstack()[ff_cols].dropna(axis=0)
    ff_cov = ff_cov_df.loc[date].loc[ff_cols,ff_cols]
    spec_cov = spec_est.loc[date]
    
    idx = spec_cov.index.intersection(betas.index)
    spec_cov = spec_cov.reindex(idx)
    betas = betas.reindex(idx)

    cov = betas@ff_cov@betas.T + np.diag(spec_cov)
    return cov

        date      0      1      2      3      4      5      6      7      8  \
0 1970-01-30  12490  10401  12079  11754  11850  14322  27983  14736  12060   
1 1970-02-27  12490  10401  12079  11754  11850  14322  14736  27983  12060   
2 1970-03-31  12490  10401  12079  11850  11754  14322  14736  27983  12060   
3 1970-04-30  12490  10401  12079  11850  11754  14322  14736  27983  12060   
4 1970-05-29  12490  10401  12079  11850  11754  14322  14736  27983  12060   

   ...   1990   1991   1992   1993   1994   1995   1996   1997   1998   1999  
0  ...  48661  31480  41304  40256  48573  48899  45460  35246  44177  37444  
1  ...  43174  46711  34331  49998  35051  41304  43406  46519  45823  38340  
2  ...  33582  46500  35641  49429  37612  36644  36695  33830  44943  41574  
3  ...  47618  31106  46498  44804  32459  47407  31819  41574  37989  47685  
4  ...  31595  32440  31106  14197  42876  44513  32133  39132  48848  43537  

[5 rows x 2001 columns]
date        permno
1970-01

In [3]:
def FactorMimickingPortOpt_MV(alpha, beta, gamma, V):
    '''
    output the optimal weights for the factor mimicking portfolio
    - only constraint is beta neutral 
    '''
    n = V.shape[0]
    x = cp.Variable(n)
    
    obj = cp.Maximize(x.T@alpha - gamma/2*cp.quad_form(x,V))
    constraints = [x.T@beta == 0]
    prob = cp.Problem(obj, constraints)
    prob.solve(solver=cp.GUROBI)
    return x.value, prob.value

def FactorMimickingPortOpt_SR(alpha, beta, V):
    '''
    output the optimal weights for the factor mimicking portfolio
    - only constraint is beta neutral 
    '''
    n = V.shape[0]
    z = cp.Variable(n)
    kappa = cp.Variable(1)

    obj = cp.Minimize(cp.quad_form(z,V))
    #constraints = [z.T@beta == 0, z.T@alpha == 1, np.ones(n)@z-kappa==0, kappa>=0, ]
    constraints = [z.T@beta == 0, z.T@alpha == 1 ]
    #constraints = [z.T@alpha == 1, kappa>=0, ]
    prob = cp.Problem(obj, constraints)
    prob.solve(solver='MOSEK')
    #return z.value/ kappa.value 
    return z.value
    


In [ ]:
targetVol = 0.4
fullData = dict()
noConv = 0
#for i in range(1, idx['date'].shape[0]):
for i in range(1, 10):
    start = time()

    #Loop through the dates
    specificDate = idx['date'][i]

    #Get the covariance matrix
    V = risk_cov(specificDate)

    #For that specific date, get the permno of the investable stocks 
    specificPermno = idx[idx['date'] == specificDate].iloc[:, 1:].T
    specificPermno.columns = ['permno']

    #Get the alphas for that specific date
    alphas = pd.DataFrame( data[specificDate]).reset_index()
    #alphas['beta_30_exalpha'] = alphas['beta_30_exalpha'] - alphas['beta_30_exalpha'].mean()

    #Get the betas for that specific date
    betas = betaData[betaData['date'] == specificDate]

    #merge the alphas of that date with the permno of the investable stocks
    alphas_permno = pd.merge(alphas, specificPermno, how = 'inner', on = 'permno')

    #merge the betas of that date with the permno of the investable stocks
    betas_permno = pd.merge(betas, specificPermno, how = 'inner', on = 'permno')

    #Merge the returns of that date with the permno of the investable stocks
    returns_permno = pd.merge(returndata[returndata['date'] == specificDate], specificPermno, how = 'inner', on = 'permno')

    #Merge, alphas, betas and returns 
    investableStocks = pd.merge(alphas_permno, betas_permno, on = 'permno', how = 'inner')
    investableStocks = pd.merge(investableStocks, returns_permno, on = 'permno', how = 'inner')

    #the shape of investable stocks is not matching with the shape of V.
    finalPermno = ''
    if V.shape[0] < investableStocks.shape[0]:
        print('V smaller')
        finalPermno = list( V.index)
        invest_copy = investableStocks.set_index('permno')
        investableStocks = invest_copy.loc[finalPermno, :]
    else:
        finalPermno = list( investableStocks.permno)
        V = V.loc[finalPermno, finalPermno]

    assert investableStocks.isna().sum().sum() == 0
    assert V.shape[0] == investableStocks.shape[0], 'Shape of alphas and V must match'

    shape = (investableStocks.shape[0], 1)
    alpha = investableStocks['beta_30_exalpha'].values.reshape(shape)
    beta = investableStocks['beta_30'].values.reshape(shape)
    V_np = V.values
    
    try:
        w = FactorMimickingPortOpt_SR(alpha, beta, V_np)
        portfolioVol = w.T @ V @ w
        scaledW = (targetVol/portfolioVol) * w

        investableStocks['w'] = w
        investableStocks['scaled_w'] = scaledW
    except:
        print('No convergence for', specificDate)
        noConv += 1
        investableStocks['w'] = np.nan
        investableStocks['scaled_w'] = np.nan
    
    fullData[specificDate] = investableStocks
    
    end = time()
    finalTime = round( (end - start)/60, 3)

    print('Iteration = {}'.format(i))
    print(finalTime, 'minutes\n')
    
print('No Convergence:', noConv)

V smaller
No convergence for 1970-02-27 00:00:00
Iteration = 1
0.956 minutes



ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\JuliJaramillo\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3444, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\JuliJaramillo\AppData\Local\Temp\ipykernel_25848\3964198668.py", line 12, in <module>
    V = risk_cov(specificDate)
  File "C:\Users\JuliJaramillo\AppData\Local\Temp\ipykernel_25848\3583142734.py", line 30, in risk_cov
    betas = ff_loadings.loc[date,(universe.loc[date],slice(None))].unstack()[ff_cols].dropna(axis=0)
  File "C:\Users\JuliJaramillo\anaconda3\lib\site-packages\pandas\core\indexing.py", line 925, in __getitem__
    return self._getitem_tuple(key)
  File "C:\Users\JuliJaramillo\anaconda3\lib\site-packages\pandas\core\indexing.py", line 1100, in _getitem_tuple
    return self._getitem_lowerdim(tup)
  File "C:\Users\JuliJaramillo\anaconda3\lib\site-packages\pandas\core\indexing.py", line 822, in _getitem_lowerdim
    return self._getitem_nested_tuple(tup

TypeError: object of type 'NoneType' has no len()

In [5]:
investableStocks[investableStocks['beta_30'] <0 ]

,beta_30_exalpha,date_x,beta_30,date_y,ret,w,scaled_w
permno,,,,,,,
47204,0.068320,1970-02-27,-0.357836,1970-02-27,-0.139785,NaN,NaN
48741,0.057206,1970-02-27,-0.193571,1970-02-27,0.012658,NaN,NaN
48979,0.123945,1970-02-27,-1.179973,1970-02-27,-0.109948,NaN,NaN
49031,0.057680,1970-02-27,-0.200567,1970-02-27,-0.050420,NaN,NaN
49066,0.060608,1970-02-27,-0.243852,1970-02-27,-0.022727,NaN,NaN
49218,0.113145,1970-02-27,-1.020344,1970-02-27,0.154412,NaN,NaN
49445,0.079269,1970-02-27,-0.519657,1970-02-27,-0.034545,NaN,NaN
49496,0.078119,1970-02-27,-0.502666,1970-02-27,-0.200000,NaN,NaN
49533,0.120219,1970-02-27,-1.124894,1970-02-27,-0.045662,NaN,NaN
